In [ ]:
# For the quantum optimizer
from qiskit_optimization.algorithms import MinimumEigenOptimizer # Related to the second item installed
from qiskit_optimization import QuadraticProgram

In [ ]:
# This is the quantum optimizer class being used for QAOA
class QuantumOptimizer:
    def __init__(self, instance, n, v):
      self.instance = instance
      self.n = n
      self.v = v

    def binary_representation(self,x_sol=0): # This function is identical to the one used by the internship and I have been given permission to use this code.
      instance = self.instance
      n = self.n
      v = self.v
      A = np.max(instance) * 100  # This is a parameter for the cost function
      # Determine the weights w
      instance_vec = instance.reshape(n ** 2)
      w_list = [instance_vec[x] for x in range(n ** 2) if instance_vec[x] > 0]
      w = np.zeros(n * (n - 1))
      for ii in range(len(w_list)):
        w[ii] = w_list[ii]
      # Some other needed variables
      Id_n = np.eye(n)
      Im_n_1 = np.ones([n - 1, n - 1])
      Iv_n_1 = np.ones(n)
      Iv_n_1[0] = 0
      Iv_n = np.ones(n-1)
      neg_Iv_n_1 = np.ones(n) - Iv_n_1
      v = np.zeros([n, n*(n-1)])
      for ii in range(n):
        count = ii-1
        for jj in range(n*(n-1)):
          if jj//(n-1) == ii:
            count = ii
          if jj//(n-1) != ii and jj%(n-1) == count:
            v[ii][jj] = 1.
        vn = np.sum(v[1:], axis=0)

        # Q defines the interactions between variables
        Q = A*(np.kron(Id_n, Im_n_1) + np.dot(v.T, v))

        # g defines the contribution from the individual variables
        g = w - 2 * A * (np.kron(Iv_n_1,Iv_n) + vn.T) - \
                2 * A * K * (np.kron(neg_Iv_n_1, Iv_n) + v[0].T)

        # c is the constant offset
        c = 2 * A * (n-1) + 2 * A * (K ** 2)

        try:
          max(x_sol)
          # Evaluates the cost distance from a binary representation of a path
          fun = lambda x: np.dot(np.around(x), np.dot(Q, np.around(x))) + np.dot(g, np.around(x)) + c
          cost = fun(x_sol)
        except:
          cost = 0

        return Q, g, c, cost # Final Note: The above function was used from the internship-based code

  # This sets up all the variables from the previous function for the final quantum algorithm being run.
    def construct_problem(self, Q, g, c) -> QuadraticProgram:
      qp = QuadraticProgram()
      for i in range(n * (n - 1)):
        qp.binary_var(str(i))
        qp.objective.quadratic = Q
        qp.objective.linear = g
        qp.objective.constant = c
      return qp

    def solve_problem(self, qp): # This function solves the function and allows one the get the optimal solution.
      algorithm_globals.random_seed = 10000 # Random number
      # This "quantum instance" code is used to run any qiskit-aqua based algorithm.
      quantum_instance = QuantumInstance(Aer.get_backend('qasm_simulator'),
                                       seed_simulator=algorithm_globals.random_seed,
                                       seed_transpiler=algorithm_globals.random_seed)
      qaoa = QAOA(optimizer = ADAM(), quantum_instance = quantum_instance, initial_point = [0,0])
      # ADAM() is a gradient-based optimizer
      # Random initial point
      optimizer = MinimumEigenOptimizer(min_eigen_solver=qaoa) # This converts the problem into a quantum unconstrained binary optimization (QUBO) problem.
      # Used to approxiate the ground state/lowest eneergy of the resulting hamiltonian
      result = optimizer.solve(qp)
      # used for computing cost of the obtained result (these two lines of code are from the internship code)
      _,_,_,level = self.binary_representation(x_sol=result.x) 
      return result.x, level